<a href="https://colab.research.google.com/github/Sagu12/all-projects/blob/master/Personaliized_news_interest_and_recommmendation_based_on_news_clickability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download("popular")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    | 

True

In [2]:
df= pd.read_json("/content/drive/MyDrive/enhanceduserdatabeta1.json", lines=True)

In [3]:
df.head()

,_index,_type,_id,_score,_source
0,enhanceduserdatabeta1,core2,AXGicw8zutugd-ZqVCZf,1,"{'publisher_id': '49491', 'date': '2020-04-22'..."
1,enhanceduserdatabeta1,core2,AXGidE5mCFpND4aqB9Bq,1,"{'publisher_id': '49491', 'date': '2020-04-22'..."
2,enhanceduserdatabeta1,core2,AXGidFqlutugd-ZqVDUg,1,"{'publisher_id': '49491', 'date': '2020-04-22'..."
3,enhanceduserdatabeta1,core2,AXGidneQutugd-ZqVE1I,1,"{'publisher_id': '49491', 'date': '2020-04-22'..."
4,enhanceduserdatabeta1,core2,AXGidvgrCFpND4aqB-8w,1,"{'publisher_id': '49491', 'date': '2020-04-22'..."


In [4]:
source=pd.json_normalize(df._source)

In [5]:
source.head(2)

,publisher_id,date,country,page_title,organisation,picture_jpg,incomelevel,mastercookie_id,clickurloriginal,sourceUrl,top_url,br_user_agent,refcurrent,refcurrentoriginal,canvas_installed,clickedurl,sessionhash,screen_resoln,audience_segment,id,state,QuarterValue,page_meta_tag,brandName,reforiginal,ip,picture_gif,font_installed,cookie_id,cookiehash,browser_name,fingerprint_id,gif_animated,localStoragecuid,system_os,subcategory,dual_orientation,channel_id,parameters,device,picture_png,streaming_mov,engagementTime,gender,city,hotspot,visitor_id,agegroup,colors,request_time,referrerType,hseries,channel_name,streaming_mp4,publisher_name,ISP,session_id,time_zone,installed_activex,resolution_properties,ux_full_desktop,referrer,modelName,streaming_video,plugin,isWireless,localStorageid,screen_properties,streaming_3gpp,browserversion,latitude_longitude,postalcode,authorName,tag,publishDatev1,section
0,49491,2020-04-22,,,None,false,medium,18fdd55d_bd79_4654_a72d_10ca6652988c,,m.aajtak.in,,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,https___m_aajtak_in_khabare_ab_tak_story_caste...,https://m.aajtak.in/khabare-ab-tak/story/caste...,2227547380,,7.683189e+17,4031165888,,1,,Quarter4,,Google,https://m.aajtak.in/khabare-ab-tak/story/caste...,"2409:4052:887:1836:1d8f:8a33:34b5:f29d, 64.233...",false,false,a81e34f6_1d23_48ca_9254_a4f6a072822d,-1.226066e+18,Chrome_Mobile,4031165888_2227547380_1146773256_420,false,1bd04fa3-e158-4560-9f12-52b318588b1f,Android_4.2,,true,78844,,Mobile,false,false,6.0,female,,hotspot,e9b7951d-f0fd-4c1b-88b3-4289eda2fb78,25_34,65536,2020-04-22 15:12:24,Others,ref,AJTK,true,NS,None,fb8adeda_347c_427b_bd09_28113082c8fc,420,1146773256,1080_1920,false,https___m_aajtak_in_khabare_ab_tak_story_caste...,Google_Nexus_5_2013_october,true,1257683291,true,a81e34f6-1d23-48ca-9254-a4f6a072822d,62_110,true,18,NaN,NaN,NaN,NaN,NaN,NaN
1,49491,2020-04-22,,,None,false,low,bf529ba0_b27a_49f9_9907_d3bd9c10ae1b,,m.aajtak.in,,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,https___m_aajtak_in_karyakrams_das_tak_video__...,https://m.aajtak.in/karyakrams/das-tak/video/-...,2227547380,,-3.215504e+18,4031165888,,1,,Quarter4,,Google,https://m.aajtak.in/,"2405:204:1299:b635:7e57:1081:c615:95d9, 64.233...",false,false,bca6391f_4a71_4591_9981_b2584d9c0b62,-5.499536e+16,Chrome_Mobile,4031165888_2227547380_1146773256_420,false,29665632-ea86-41b7-a814-17d69d4ba110,Android_4.2,,true,78844,,Mobile,false,false,1.0,male,,hotspot,754cc585-b30e-4b62-9288-9fedfdc42684,45_54,65536,2020-04-22 15:13:13,Others,ref,AJTK,true,NS,None,72c7497f_1464_4669_9c47_6bf72ad0ccf8,420,1146773256,1080_1920,false,https___m_aajtak_in_,Google_Nexus_5_2013_october,true,1257683291,true,bca6391f-4a71-4591-9981-b2584d9c0b62,62_110,true,18,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
con= pd.concat([df, source], axis=1)

In [7]:
con= con.drop("_source", axis=1)

In [8]:
con["refcurrentoriginal"]= con["refcurrentoriginal"].astype("str")

In [9]:
con['result'] = con['refcurrentoriginal'].map(lambda x: x.lstrip('https://m. www').rstrip('aAbBcC'))

In [ ]:
ps= WordNetLemmatizer()
corpus=[]
for i in range(0,len(con)):
  print(i)
  review= re.sub("[^a-zA-Z]", " ", con["result"][i])
  review= review.lower()
  review= review.split()
  review= [ps.lemmatize(word) for word in review if not word in stopwords.words("english")]
  review= " ".join(review)
  corpus.append(review)

In [12]:
con["actual_news"]= corpus

In [13]:
con.head(2)

,_index,_type,_id,_score,publisher_id,date,country,page_title,organisation,picture_jpg,incomelevel,mastercookie_id,clickurloriginal,sourceUrl,top_url,br_user_agent,refcurrent,refcurrentoriginal,canvas_installed,clickedurl,sessionhash,screen_resoln,audience_segment,id,state,QuarterValue,page_meta_tag,brandName,reforiginal,ip,picture_gif,font_installed,cookie_id,cookiehash,browser_name,fingerprint_id,gif_animated,localStoragecuid,system_os,subcategory,...,parameters,device,picture_png,streaming_mov,engagementTime,gender,city,hotspot,visitor_id,agegroup,colors,request_time,referrerType,hseries,channel_name,streaming_mp4,publisher_name,ISP,session_id,time_zone,installed_activex,resolution_properties,ux_full_desktop,referrer,modelName,streaming_video,plugin,isWireless,localStorageid,screen_properties,streaming_3gpp,browserversion,latitude_longitude,postalcode,authorName,tag,publishDatev1,section,result,actual_news
0,enhanceduserdatabeta1,core2,AXGicw8zutugd-ZqVCZf,1,49491,2020-04-22,,,None,false,medium,18fdd55d_bd79_4654_a72d_10ca6652988c,,m.aajtak.in,,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,https___m_aajtak_in_khabare_ab_tak_story_caste...,https://m.aajtak.in/khabare-ab-tak/story/caste...,2227547380,,7.683189e+17,4031165888,,1,,Quarter4,,Google,https://m.aajtak.in/khabare-ab-tak/story/caste...,"2409:4052:887:1836:1d8f:8a33:34b5:f29d, 64.233...",false,false,a81e34f6_1d23_48ca_9254_a4f6a072822d,-1.226066e+18,Chrome_Mobile,4031165888_2227547380_1146773256_420,false,1bd04fa3-e158-4560-9f12-52b318588b1f,Android_4.2,,...,,Mobile,false,false,6.0,female,,hotspot,e9b7951d-f0fd-4c1b-88b3-4289eda2fb78,25_34,65536,2020-04-22 15:12:24,Others,ref,AJTK,true,NS,None,fb8adeda_347c_427b_bd09_28113082c8fc,420,1146773256,1080_1920,false,https___m_aajtak_in_khabare_ab_tak_story_caste...,Google_Nexus_5_2013_october,true,1257683291,true,a81e34f6-1d23-48ca-9254-a4f6a072822d,62_110,true,18,NaN,NaN,NaN,NaN,NaN,NaN,aajtak.in/khabare-ab-tak/story/caste-based-cen...,aajtak khabare ab tak story caste based census...
1,enhanceduserdatabeta1,core2,AXGidE5mCFpND4aqB9Bq,1,49491,2020-04-22,,,None,false,low,bf529ba0_b27a_49f9_9907_d3bd9c10ae1b,,m.aajtak.in,,Mozilla/5.0 (Linux; Android 4.2.1; en-us; Nexu...,https___m_aajtak_in_karyakrams_das_tak_video__...,https://m.aajtak.in/karyakrams/das-tak/video/-...,2227547380,,-3.215504e+18,4031165888,,1,,Quarter4,,Google,https://m.aajtak.in/,"2405:204:1299:b635:7e57:1081:c615:95d9, 64.233...",false,false,bca6391f_4a71_4591_9981_b2584d9c0b62,-5.499536e+16,Chrome_Mobile,4031165888_2227547380_1146773256_420,false,29665632-ea86-41b7-a814-17d69d4ba110,Android_4.2,,...,,Mobile,false,false,1.0,male,,hotspot,754cc585-b30e-4b62-9288-9fedfdc42684,45_54,65536,2020-04-22 15:13:13,Others,ref,AJTK,true,NS,None,72c7497f_1464_4669_9c47_6bf72ad0ccf8,420,1146773256,1080_1920,false,https___m_aajtak_in_,Google_Nexus_5_2013_october,true,1257683291,true,bca6391f-4a71-4591-9981-b2584d9c0b62,62_110,true,18,NaN,NaN,NaN,NaN,NaN,NaN,aajtak.in/karyakrams/das-tak/video/-when-will-...,aajtak karyakrams da tak video crime woman stop


In [14]:
con.drop(["result", "refcurrentoriginal"], axis=1, inplace=True)

In [27]:
con.shape

(107600, 80)

In [21]:
con_updated= con[["ip","engagementTime","gender","visitor_id","actual_news","audience_segment"]]

In [22]:
con_updated.head(2)

,ip,engagementTime,gender,visitor_id,actual_news,audience_segment
0,"2409:4052:887:1836:1d8f:8a33:34b5:f29d, 64.233...",6.0,female,e9b7951d-f0fd-4c1b-88b3-4289eda2fb78,aajtak khabare ab tak story caste based census...,
1,"2405:204:1299:b635:7e57:1081:c615:95d9, 64.233...",1.0,male,754cc585-b30e-4b62-9288-9fedfdc42684,aajtak karyakrams da tak video crime woman stop,


In [23]:
con_updated.isnull().sum()

ip                     0
engagementTime      8116
gender              8129
visitor_id             0
actual_news            0
audience_segment       0
dtype: int64

In [24]:
con_updated= con_updated[con_updated.actual_news != "nan"]

In [26]:
con_updated.isna().sum()

ip                  0
engagementTime      0
gender              0
visitor_id          0
actual_news         0
audience_segment    0
dtype: int64

In [28]:
con_updated.shape

(99471, 6)

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectornews = vectorizer.fit_transform(con_updated["actual_news"])

In [31]:
from sklearn.decomposition import LatentDirichletAllocation

In [32]:
LDA = LatentDirichletAllocation(n_components=23, random_state=42)

In [33]:
topic_values = LDA.fit_transform(vectornews)

In [34]:
con_updated['audience_segment_updated'] = topic_values.argmax(axis=1)

In [35]:
con_updated.head(2)

,ip,engagementTime,gender,visitor_id,actual_news,audience_segment,audience_segment_updated
0,"2409:4052:887:1836:1d8f:8a33:34b5:f29d, 64.233...",6.0,female,e9b7951d-f0fd-4c1b-88b3-4289eda2fb78,aajtak khabare ab tak story caste based census...,,21
1,"2405:204:1299:b635:7e57:1081:c615:95d9, 64.233...",1.0,male,754cc585-b30e-4b62-9288-9fedfdc42684,aajtak karyakrams da tak video crime woman stop,,21


In [40]:
import numpy as np  

In [100]:
data= con_updated.groupby(['ip', 'audience_segment_updated']).size().unstack().fillna(0)


In [101]:
data

audience_segment_updated,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
ip,,,,,,,,,,,,,,,,,,,,,,,
1.120.96.87,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.126.105.168,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.127.105.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1.127.109.114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1.129.107.192,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.40.239.168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99.46.72.90,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99.6.202.121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
data[0].values

array([0., 0., 0., ..., 0., 0., 0.])

In [103]:
data.dtypes

audience_segment_updated
0     float64
1     float64
2     float64
3     float64
4     float64
5     float64
6     float64
7     float64
8     float64
9     float64
10    float64
11    float64
12    float64
13    float64
14    float64
15    float64
16    float64
17    float64
18    float64
19    float64
20    float64
21    float64
22    float64
dtype: object

In [104]:
first=data[0].values
second=data[1].values
third=data[2].values
fourth=data[3].values
fifth=data[4].values
sixth=data[5].values
seventh=data[6].values
eighth=data[7].values
ninth=data[8].values
tenth=data[9].values
eleventh=data[10].values
twelveth=data[11].values
thirteenth=data[12].values
fourteenth=data[13].values
fifteenth=data[14].values
sixteenth=data[15].values
seventeenth=data[16].values
eighteenth=data[17].values
ninteenth=data[18].values
twentieth=data[19].values
twentyone=data[20].values
twentytwo=data[21].values
twentythree=data[22].values
#twentyfour=data[23].values

In [105]:
Total= first+second+third+fourth+fifth+sixth+seventh+eighth+ninth+tenth+eleventh+twelveth+thirteenth+fourteenth+fifteenth+sixteenth+seventeenth+eighteenth+ninteenth+twentieth+twentyone+twentytwo+twentythree

In [106]:
data['Total']=Total

In [107]:
data.Total.value_counts()

1.0      81662
2.0       5251
3.0        790
4.0        192
5.0         79
6.0         49
7.0         18
8.0         16
10.0        10
9.0          9
17.0         5
11.0         5
18.0         3
20.0         3
16.0         3
24.0         3
22.0         2
63.0         2
13.0         2
38.0         2
52.0         1
67.0         1
29.0         1
34.0         1
33.0         1
25.0         1
48.0         1
41.0         1
79.0         1
26.0         1
19.0         1
70.0         1
14.0         1
23.0         1
78.0         1
709.0        1
36.0         1
53.0         1
576.0        1
144.0        1
27.0         1
12.0         1
109.0        1
55.0         1
40.0         1
Name: Total, dtype: int64

In [108]:
data['firstbytotal']= first/Total
data['secondbytotal']= second/Total
data['thirdbytotal']= third/Total
data['fourthbytotal']= fourth/Total
data['fifthbytotal']= fifth/Total
data['sixthbytotal']= sixth/Total
data['seventhbytotal']= seventh/Total
data['eighthbytotal']= eighth/Total
data['ninthbytotal']= ninth/Total
data['tenthbytotal']= tenth/Total
data['elevbytotal']= eleventh/Total
data['twelvbytotal']= twelveth/Total
data['thirteenbytotal']= thirteenth/Total
data['fourteenbytotal']= fourteenth/Total
data['fifteenbytotal']= fifteenth/Total
data['sixteenbytotal']= sixteenth/Total
data['seventeenbytotal']= seventeenth/Total
data['eighteenbytotal']= eighteenth/Total
data['ninteenbytotal']= ninteenth/Total
data['twentybytotal']= twentieth/Total
data['twentyonebytotal']= twentyone/Total
data['twentytwobytotal']= twentytwo/Total
data['twentythreebytotal']= twentythree/Total
#data['twentyfourbytotal']= twentyfour/Total

In [109]:
totalSum=np.sum(data['Total'])
print(totalSum)

99471.0


In [110]:
data['PtClick']=data['Total']/totalSum

In [111]:
data.head()

audience_segment_updated,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,Total,firstbytotal,secondbytotal,thirdbytotal,fourthbytotal,fifthbytotal,sixthbytotal,seventhbytotal,eighthbytotal,ninthbytotal,tenthbytotal,elevbytotal,twelvbytotal,thirteenbytotal,fourteenbytotal,fifteenbytotal,sixteenbytotal,seventeenbytotal,eighteenbytotal,ninteenbytotal,twentybytotal,twentyonebytotal,twentytwobytotal,twentythreebytotal,PtClick
ip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.120.96.87,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00001
1.126.105.168,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00001
1.127.105.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00001
1.127.109.114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00001
1.129.107.192,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00001


In [113]:
#taking equal chances of getting different segments of articles so our constant according to assumption will be 0.043 as per 1/23 output

data["interestfirst"]= (data.firstbytotal * data.PtClick) / 0.043
data["interestsecond"]= (data.secondbytotal * data.PtClick) / 0.043
data["interestthird"]= (data.thirdbytotal * data.PtClick) / 0.043
data["interestfourth"]= (data.fourthbytotal * data.PtClick) / 0.043
data["interestfifth"]= (data.fifthbytotal * data.PtClick) / 0.043
data["interestsixth"]= (data.sixthbytotal * data.PtClick) / 0.043
data["interestseventh"]= (data.seventhbytotal * data.PtClick) / 0.043
data["interesteighth"]= (data.eighthbytotal * data.PtClick) / 0.043
data["interestninth"]= (data.ninthbytotal * data.PtClick) / 0.043
data["interesttenth"]= (data.tenthbytotal * data.PtClick) / 0.043
data["interesteleven"]= (data.elevbytotal * data.PtClick) / 0.043
data["interesttwelve"]= (data.twelvbytotal * data.PtClick) / 0.043
data["interestthirteen"]= (data.thirteenbytotal * data.PtClick) / 0.043
data["interestfourteen"]= (data.fourteenbytotal * data.PtClick) / 0.043
data["interestfifteen"]= (data.fifteenbytotal * data.PtClick) / 0.043
data["interestsixteen"]= (data.sixteenbytotal * data.PtClick) / 0.043
data["interestseventeen"]= (data.seventeenbytotal * data.PtClick) / 0.043
data["interesteighteen"]= (data.eighteenbytotal * data.PtClick) / 0.043
data["interestninteen"]= (data.ninteenbytotal * data.PtClick) / 0.043
data["interesttwenty"]= (data.twentybytotal * data.PtClick) / 0.043
data["interesttwentyone"]= (data.twentyonebytotal * data.PtClick) / 0.043
data["interesttwentytwo"]= (data.twentytwobytotal * data.PtClick) / 0.043
data["interesttwentythree"]= (data.twentythreebytotal * data.PtClick) / 0.043

In [115]:
data

audience_segment_updated,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,Total,firstbytotal,secondbytotal,thirdbytotal,fourthbytotal,fifthbytotal,sixthbytotal,seventhbytotal,eighthbytotal,ninthbytotal,tenthbytotal,elevbytotal,twelvbytotal,thirteenbytotal,fourteenbytotal,fifteenbytotal,sixteenbytotal,seventeenbytotal,eighteenbytotal,ninteenbytotal,twentybytotal,twentyonebytotal,twentytwobytotal,twentythreebytotal,PtClick,interestfirst,interestsecond,interestthird,interestfourth,interestfifth,interestsixth,interestseventh,interesteighth,interestninth,interesttenth,interesteleven,interesttwelve,interestthirteen,interestfourteen,interestfifteen,interestsixteen,interestseventeen,interesteighteen,interestninteen,interesttwenty,interesttwentyone,interesttwentytwo,interesttwentythree
ip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.120.96.87,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00001,0.0,0.0,0.0,0.0,0.000000,0.000234,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1.126.105.168,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00001,0.0,0.0,0.0,0.0,0.000000,0.000234,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1.127.105.71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00001,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000234,0.0,0.0,0.0
1.127.109.114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.00001,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000234,0.0,0.0,0.0
1.129.107.192,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00001,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000234,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.40.239.168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00001,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
99.46.72.90,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00002,0.0,0.0,0.0,0.0,0.000234,0.000234,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
99.6.202.121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00001,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


## The above data represents the interest of individual user on different segments of news based on the clickability of the same.

In [116]:
data.to_csv("click_interest.csv")